In [1]:
import tweepy
from tweepy import OAuthHandler, Stream

In [9]:
from tweepy.streaming import Stream
import socket
import json

In [1]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

In [ ]:
class TweetsListener(Stream):

    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            msg = json.loads( data )
            print( msg['text'].encode('utf-8') )
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True

In [ ]:
def sendData(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=['soccer'])

In [ ]:
if __name__ == "__main__":
    s = socket.socket()         # Create a socket object
    host = "127.0.0.1"     # Get local machine name
    port = 5555                 # Reserve a port for your service.
    s.bind((host, port))        # Bind to the port

    print("Listening on port: %s" % str(port))

    s.listen(5)                 # Now wait for client connection.
    c, addr = s.accept()        # Establish connection with client.

    print( "Received request from: " + str( addr ) )

    sendData( c )

In [2]:
import tweepy

# Set up your credentials

# Authenticate with Twitter API
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_secret)
api = tweepy.API(auth)

# Check authentication
try:
    user = api.verify_credentials()
    print('Authentication successful. User:', user.screen_name)
except tweepy.TweepError as e:
    print('Authentication error:', e)


Authentication successful. User: Danial660099


In [1]:
import findspark

In [2]:
findspark.init('/home/danial/spark-3.3.2-bin-hadoop3')

In [3]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

23/04/19 13:42:13 WARN Utils: Your hostname, danial-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/19 13:42:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/19 13:42:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [7]:
lines = socket_stream.window( 20 )

In [8]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [9]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

In [10]:
ssc.start() 

23/04/19 13:42:31 ERROR JobScheduler: Error running job streaming job 1681936950000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_4840/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 102, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 894, in createDataFrame
    return self._create_dataframe(
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session

23/04/19 13:42:40 ERROR JobScheduler: Error running job streaming job 1681936960000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_4840/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 102, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 894, in createDataFrame
    return self._create_dataframe(
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session

23/04/19 13:42:50 ERROR JobScheduler: Error running job streaming job 1681936970000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_4840/1757346720.py", line 7, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 102, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session.py", line 894, in createDataFrame
    return self._create_dataframe(
  File "/home/danial/spark-3.3.2-bin-hadoop3/python/pyspark/sql/session